Import Classes

In [1]:
import copy

Block Class

In [2]:
class block:
    
    def __init__(self, block_id, property_name = None, value_string = None):                             
        self.block_id = block_id
        self.property_name = property_name
        self.value_string = value_string
        self.on_top_of = None
        self.side_by_side = []
        self.below_of = None
        
    def __lt__(self, other):
         return self.block_id < other.block_id
        
    def __eq__(self, other):
        if(other == "table" or other == None):
            return False
        else:
            return self.block_id == other.block_id and ((self.on_top_of.block_id if self.on_top_of != None else None) == \
            (other.on_top_of.block_id if other.on_top_of != None else None)) and \
            [x.block_id for x in self.side_by_side] == [x.block_id for x in other.side_by_side]
        
    def __ne__(self, other):
        if(other == "table" or other == None):
            return True
        else:
            return self.block_id != other.block_id and ((self.on_top_of.block_id if self.on_top_of != None else None) !=  \
            (other.on_top_of.block_id if other.on_top_of != None else None)) and \
            [x.block_id for x in self.side_by_side] != [x.block_id for x in other.side_by_side]
    
    def set_on_top_of(self, block_one):
        if(self.on_top_of == None):
            self.on_top_of = block_one
            block_one.below_of = self
        else:
            print("Only one block can be placed on top of a block")
            
    def set_side_by_side(self, block_one):
        if(block_one in self.side_by_side):
            print(block_one.block_id + " is already side by side with " + self.block_id)
        else:
            self.side_by_side.append(block_one)
            self.side_by_side = sorted(self.side_by_side)
            
    def remove_on_top_of(self):
        self.on_top_of.below_of = None
        self.on_top_of = None
        
    def remove_side_by_side(self):
        for w in self.side_by_side:
            w.side_by_side.remove(self)
        self.side_by_side = []
        
    def __repr__(self):
        return "{" + self.block_id + ", (" + ("None", self.property_name)[self.property_name != None] + ": " + \
        ("None", self.value_string)[self.value_string != None] + "), (on-top-of: " + \
        ("None" if self.on_top_of == None else self.on_top_of.block_id) + "), (side-by-side: " + \
        str([x.block_id for x in self.side_by_side]) + ")}\n"
        
    def __str__(self):
        return "{" + self.block_id + ", (" + ("None", self.property_name)[self.property_name != None] + ": " + \
        ("None", self.value_string)[self.value_string != None] + "), (on-top-of: " + \
        ("None" if self.on_top_of == None else self.on_top_of.block_id) + "), (side-by-side: " + \
        str([x.block_id for x in self.side_by_side]) + ")}\n"

Slide-To Function

In [3]:
def slide_to(block_one, block_two, command = True):
    if(slide_to_check(block_one, block_two, command, True)):
        z = block_one
        if(command):block_one.remove_side_by_side()
        while(z.on_top_of != None and command):
            z = z.on_top_of
            z.remove_side_by_side()
        block_one.set_side_by_side(block_two)
        block_two.set_side_by_side(block_one)
        x = block_one
        y = block_two
        
        while(x.on_top_of != None and y.on_top_of != None):
            x = x.on_top_of
            y = y.on_top_of
            x.set_side_by_side(y)
            y.set_side_by_side(x)

Slide-To-Check Function checks pre-conditions for Slide-To Function

In [4]:
def slide_to_check(block_one, block_two, command = True, verbose = False):
    if((block_one.below_of != None) and command):
        if(verbose): print(block_one.block_id + " height is not zero")
        return False
    elif((block_two.below_of != None) and command):
        if(verbose): print(block_two.block_id + " height is not zero")
        return False
    elif(len(block_two.side_by_side) >= 4):
        if(verbose): print(block_one.block_id + " already has four block side-by-side with it")
        return False
    elif(block_one in block_two.side_by_side):
        if(verbose): print(block_one.block_id + " is already side by side with " + block_two.block_id)
        return False
    else:
        return True

Stack Function

In [5]:
def stack(block_one, block_two, command = True):
    if(block_one.on_top_of != None and command):
        print(block_one.block_id + " already has a block on top of it")
    elif(block_two == "table" or block_two == None):
        block_one.remove_side_by_side()
        if(block_one.below_of != None): block_one.below_of.on_top_of = None
        block_one.below_of = None
        a = block_one
    elif(block_two.on_top_of != None and command):
        print(block_two.block_id + " already has a block on top of it")
    else:
        if(block_one.below_of != None): block_one.below_of.on_top_of = None
        block_one.remove_side_by_side()
        block_one.below_of = block_two
        block_two.on_top_of = block_one
        if(len(block_two.side_by_side) > 0):
            for z in block_two.side_by_side:
                x = z
                y = block_two
                while(x.on_top_of != None and y.on_top_of != None):
                    x = x.on_top_of
                    y = y.on_top_of
                    x.set_side_by_side(y)
                    y.set_side_by_side(x)

Action Function returns all possible actions from input state

In [784]:
def actions(state):
    actions = []
    slides = []
    stacks = []
    for indz, z in enumerate(state):
        if(z.on_top_of == None and (len(z.side_by_side) != 0 or z.below_of != None)):
            
            actions.append(("stack", (indz, z.block_id), None))
    for indx, x in enumerate(state):
        for indy, y in enumerate(state):
            if(x.block_id != y.block_id):
                if(slide_to_check(x, y) == True and (len(x.side_by_side) > 0 or len(y.side_by_side) > 0)):
                    slides.append(("slide-to", (indx, x.block_id), (indy, y.block_id)))
                if(slide_to_check(x, y) == True and (len(x.side_by_side) == 0 and len(y.side_by_side) == 0) and x < y):
                    slides.append(("slide-to", (indx, x.block_id), (indy, y.block_id)))
                if(x.on_top_of == None and y.on_top_of == None):
                    stacks.append(("stack", (indx, x.block_id), (indy, y.block_id)))
    
    return actions + slides + stacks

In [785]:
startState = read_startState("case4start.txt") #Input File
print(startState)
actions(startState)

[{block1, (color: red), (on-top-of: block2), (side-by-side: [])}
, {block10, (color: red), (on-top-of: block11), (side-by-side: [])}
, {block11, (color: blue), (on-top-of: block12), (side-by-side: [])}
, {block12, (color: blue), (on-top-of: block13), (side-by-side: [])}
, {block13, (color: green), (on-top-of: block14), (side-by-side: [])}
, {block14, (color: green), (on-top-of: None), (side-by-side: [])}
, {block15, (color: green), (on-top-of: None), (side-by-side: [])}
, {block2, (color: blue), (on-top-of: block3), (side-by-side: [])}
, {block3, (color: green), (on-top-of: block4), (side-by-side: [])}
, {block4, (color: black), (on-top-of: block5), (side-by-side: [])}
, {block5, (color: white), (on-top-of: block6), (side-by-side: [])}
, {block6, (color: yellow), (on-top-of: block7), (side-by-side: [])}
, {block7, (color: yellow), (on-top-of: None), (side-by-side: [])}
, {block8, (color: yellow), (on-top-of: block9), (side-by-side: [])}
, {block9, (color: red), (on-top-of: block10), (s

[('stack', (5, 'block14'), None),
 ('stack', (12, 'block7'), None),
 ('slide-to', (0, 'block1'), (6, 'block15')),
 ('slide-to', (0, 'block1'), (13, 'block8')),
 ('slide-to', (6, 'block15'), (13, 'block8')),
 ('stack', (5, 'block14'), (6, 'block15')),
 ('stack', (5, 'block14'), (12, 'block7')),
 ('stack', (6, 'block15'), (5, 'block14')),
 ('stack', (6, 'block15'), (12, 'block7')),
 ('stack', (12, 'block7'), (5, 'block14')),
 ('stack', (12, 'block7'), (6, 'block15'))]

Take-Action Function implements action from Action Function on input state

In [7]:
def take_action(state, action):
    newState = copy.deepcopy(state)
    if(action[0] == "slide-to"):
        slide_to(newState[action[1][0]], newState[action[2][0]])
    if(action[0] == "stack"):
        stack(newState[action[1][0]], newState[action[2][0]] if action[2] != None else None)
    return newState

Read-File Function takes input file name and returns state

In [8]:
def read_startState(file_name):
    f = list(open(file_name, "r"))
    state = []
    for x in f:
        if(len(x.split()) == 4):
            ary = x.replace("(", "").replace(")", "").split()
            if(ary[0] == "has"):
                new_block = block(ary[1], ary[2], ary[3])
                state.append(new_block)
    for y in f:
        if(len(y.split()) == 4):
            ary = y.replace("(", "").replace(")", "").split()
            if(ary[0] == "is"):
                list_one = [a for a in state if a.block_id == ary[1]]
                if(len(list_one) > 0):
                    block_one = list_one[0]
                else:
                    block_one = block(ary[1])
                    state.append(block_one)
                if(ary[2] == "table"):
                    block_two = None
                else:
                    list_two = [b for b in state if b.block_id == ary[2]]
                    if(len(list_two) > 0):
                        block_two = list_two[0]
                    else:
                        block_two = block(ary[2])
                        state.append(block_two)
                if(ary[3] == "on-top-of"):
                    stack(block_one, block_two, command = False)
                elif(ary[3] == "side-by-side"):
                    slide_to(block_one, block_two, command = False)
    return sorted(state)

In [9]:
def read_goalState(file_name):
    return list(open(file_name, "r"))

In [10]:
def check(state, goalState):
    for x in goalState:
        if(len(x.split()) == 4):
            ary = x.replace("(", "").replace(")", "").split()
            state_block_ids = [y.block_id for y in state]
            if(ary[0] == "has"):
                if(ary[1] not in state_block_ids):
                    return False
            if(ary[0] == "is"):
                if(ary[3] == "on-top-of"):
                    if(ary[2] == 'table' and (ary[1] not in state_block_ids)):
                        print("Block in goal isn't in state")
                        return False
                    elif((ary[2] not in state_block_ids) or (ary[1] not in state_block_ids)):
                        print("Block in goal isn't in state")
                        return False
                    if(ary[2] == 'table'):
                        for w in state:
                            if(w.block_id == ary[1] and w.below_of != None):
                                return False
                    else:
                        for z in state:
                            if(z.block_id == ary[2] and (z.on_top_of.block_id if z.on_top_of != None else None) != ary[1]):
                                return False

                if(ary[3] == "side-by-side"):
                    if((ary[2] not in state_block_ids) or (ary[1] not in state_block_ids)):
                        print("Block in goal isn't in state")
                        return False
                    for z in state:
                        if(z.block_id == ary[2] and (ary[1] not in [w.block_id for w in z.side_by_side])):
                            return False
    return True

Simple-Heuristic Function always returns 1. Ignore h_1 and h_2. h_3 is the fastest heuristic and doesn't overestimate.

In [11]:
def h_simple(state, goal):
    return 1

In [12]:
def h_1(state, goal):
    cost = 0
    for indx, x in enumerate(state):
        y = goal[indx]
        if(x.on_top_of != y.on_top_of):
            cost = cost + 1
        for z in x.side_by_side:
            if(z not in y.side_by_side):
                cost = cost + 0.5
    return cost

In [13]:
def h_2(state, goal):
    cost = 0
    dic = {}
    for indx, x in enumerate(state):
        y = goal[indx]
        if((x.below_of.block_id if x.below_of != None else None) != (y.below_of.block_id if y.below_of != None else None)):
            cost = cost + 1
                        
    state2, goal2 = zip(*sorted(zip(state, goal), key = lambda x: len(x[0].side_by_side), reverse=True))
    for indv, v in enumerate(state2):
        for u in v.side_by_side:
            if(dic.get((v.block_id, u.block_id), 0) == 0 and dic.get((u.block_id, v.block_id), 0) == 0):
                if(u.block_id not in [n.block_id for n in goal2[indv].side_by_side]):
                    cost = cost + 1
                    for t in v.side_by_side:
                        dic[(v.block_id, t.block_id)] = 1
                    a = v
                    while(a.on_top_of != None):
                        a = a.on_top_of
                        for s in a.side_by_side:
                            dic[(s.block_id, a.block_id)] = 1
                    a = v
                    while(a.below_of != None):
                        a = a.below_of
                        for r in a.side_by_side:
                            dic[(r.block_id, a.block_id)] = 1
    return cost

In [14]:
def h_3(state, goal):
    cost = 0
    dic = {}
    for indx, x in enumerate(state):
        y = goal[indx]
        if((x.below_of.block_id if x.below_of != None else None) != (y.below_of.block_id if y.below_of != None else None)):
            cost = cost + 1
        
    for indz, z in enumerate(goal):
        for w in z.side_by_side:
            if(dic.get((w.block_id, z.block_id), 0) == 0 and dic.get((z.block_id, w.block_id), 0) == 0 and z.below_of == None):
                if(w.block_id not in [m.block_id for m in state[indz].side_by_side]):
                    cost = cost + 1
                    dic[(w.block_id, z.block_id)] = 1
    return cost

In [16]:
def h_4(state, goal):
    cost = 0
    dic = {}
    dic2 = {}
    
    for x in goalState:
        ary = x.replace("(", "").replace(")", "").split()
        if(len(x.split()) == 4):
            if(ary[0] == "is"):
                if(ary[3] == "on-top-of"):
                    if(ary[2] == 'table'):
                        for w in state:
                            if(w.block_id == ary[1] and w.below_of != None):
                                cost = cost + 1
                                dic2[(ary[1], ary[2])] = 1
                    else:
                        for z in state:
                            if(z.block_id == ary[2] and (z.on_top_of.block_id if z.on_top_of != None else None) != ary[1]):
                                cost = cost + 1
                                dic2[(ary[1], ary[2])] = 1
    for x in goalState:
        ary = x.replace("(", "").replace(")", "").split()
        if(len(x.split()) == 4):
            if(ary[0] == "is"):
                if(ary[3] == "side-by-side"):
                    
                    for z in state:
                        if(dic.get((ary[1], ary[2]), 0) == 0 and dic.get((ary[2], ary[1]), 0) == 0 and z.block_id == ary[1] and (ary[2] not in [w.block_id for w in z.side_by_side])):
                            cost = cost + 1
                            dic[(ary[1], ary[2])] = 1
                            a = z
                            b = next((x for x in state if x.block_id == ary[2]), None)
                            while(a.on_top_of != None and b.on_top_of != None):
                                a = a.on_top_of
                                b = b.on_top_of
                                dic[(a.block_id, b.block_id)] = 1 
                            a = z
                            b = next((x for x in state if x.block_id == ary[1]), None)
                            while(a.below_of != None and b.below_of != None):
                                a = a.below_of
                                b = b.below_of
                                dic[(a.block_id, b.block_id)] = 1
                            if(dic2.get((ary[1], ary[2]), 0) == 0 and dic2.get((ary[2], ary[1]), 0) == 0):
                                block_one = next((g for g in state if g.block_id == ary[1]), None)
                                block_two = next((f for f in state if f.block_id == ary[2]), None)
                                if_sided = False
                                block_one_height = 0
                                block_two_height = 0
                                while(block_one.below_of != None):
                                    block_one = block_one.below_of
                                    if(len(block_one.side_by_side) > 0): if_sided = True
                                    block_one_height = block_one_height + 1
                                while(block_two.below_of != None):
                                    block_two = block_two.below_of
                                    if(len(block_two.side_by_side) > 0): if_sided = True
                                    block_two_height = block_two_height + 1
                                if(block_two_height != block_one_height and not if_sided and not(block_two_height == 0 or block_one_height == 0)):
                                    cost = cost + 1
    return cost

2


In [ ]:
def h_5(state, goal):
    cost = 0
    dic = {}
    
    for x in goalState:
        ary = x.replace("(", "").replace(")", "").split()
        if(len(x.split()) == 4):
            if(ary[0] == "is"):
                if(ary[3] == "on-top-of"):
                    for w in state:
                        if(w.block_id == ary[1] and (w.below_of.block_id if w.below_of != None else None) != (ary[2] if ary[2] != 'table' else None)):
                            cost = cost + 1
    for x in goalState:
        ary = x.replace("(", "").replace(")", "").split()
        if(len(x.split()) == 4):
            if(ary[0] == "is"):
                if(ary[3] == "side-by-side"):
                    block_one = next((g for g in state if g.block_id == ary[1]), None)
                    block_two = next((f for f in state if f.block_id == ary[2]), None)
                    while(block_one.below_of != None):
                        block_one = block_one.below_of
                        if(block_one.block_id == ary[2]):
                            cost = cost + 1
                    while(block_two.below_of != None):
                        block_two = block_two.below_of
                        if(block_two.block_id == ary[1]):
                            cost = cost + 1
                    for z in state:
                        if(dic.get((ary[1], ary[2]), 0) == 0 and dic.get((ary[2], ary[1]), 0) == 0 and z.block_id == ary[1] and (ary[2] not in [w.block_id for w in z.side_by_side])):
                            cost = cost + 1
                            dic[(ary[1], ary[2])] = 1
                            a = z
                            b = next((x for x in state if x.block_id == ary[2]), None)
                            while(a.on_top_of != None and b.on_top_of != None):
                                a = a.on_top_of
                                b = b.on_top_of
                                dic[(a.block_id, b.block_id)] = 1 
                            a = z
                            b = next((x for x in state if x.block_id == ary[1]), None)
                            while(a.below_of != None and b.below_of != None):
                                a = a.below_of
                                b = b.below_of
                                dic[(a.block_id, b.block_id)] = 1
                            
    return cost

In [304]:
def h_6(state, goal):
    cost = 0
    dic = {}
    dic2 = {}
    
    for x in goalState:
        ary = x.replace("(", "").replace(")", "").split()
        if(len(x.split()) == 4):
            if(ary[0] == "is"):
                if(ary[3] == "on-top-of"):
                    block_one = next((g for g in state if g.block_id == ary[1]), None)
                    block_two = next((f for f in state if f.block_id == ary[2]), None)
                    if(dic2.get((ary[1], ary[2]), 0) == 0 and (block_one.below_of.block_id if block_one.below_of != None else None) != (ary[2] if ary[2] != 'table' else None)):
                        cost = cost + 1
                        dic2[(ary[1], block_one.below_of.block_id if block_one.below_of != None else None)] = 1
                        while(block_one.on_top_of != None and dic2.get((block_one.on_top_of.block_id, block_one.block_id), 0) == 0):
                            cost = cost + 1
                            dic2[(block_one.on_top_of.block_id, block_one.block_id)] = 1
                            block_one = block_one.on_top_of
                        while(block_two.on_top_of != None and dic2.get((block_two.on_top_of.block_id, block_two.block_id), 0) == 0):
                            cost = cost + 1
                            dic2[(block_two.on_top_of.block_id, block_two.block_id)] = 1
                            block_two = block_two.on_top_of
                        
    for x in goalState:
        ary = x.replace("(", "").replace(")", "").split()
        if(len(x.split()) == 4):
            if(ary[0] == "is"):
                if(ary[3] == "side-by-side"):
                    block_one = next((g for g in state if g.block_id == ary[1]), None)
                    block_two = next((f for f in state if f.block_id == ary[2]), None)
                    if(block_one.block_id not in [c.block_id for c in block_two.side_by_side]):
                        one = copy.deepcopy(block_one)
                        one_height = 0
                        two = copy.deepcopy(block_two)
                        two_height = 0
                        while(one.below_of != None):
                            one = one.below_of
                            one_height = one_height + 1
                        while(two.below_of != None):
                            two = two.below_of
                            two_height = two_height + 1
                        if(one_height != two_height):
                            cost = cost + 1
                        elif(dic.get((ary[1], ary[2]), 0) == 0 and dic.get((ary[2], ary[1]), 0) == 0):
                            cost = cost + 1
                            dic[(ary[1], ary[2])] = 1
                            a = copy.deepcopy(block_one)
                            b = copy.deepcopy(block_two)
                            while(a.on_top_of != None and b.on_top_of != None):
                                a = a.on_top_of
                                b = b.on_top_of
                                dic[(a.block_id, b.block_id)] = 1 
                            a = copy.deepcopy(block_one)
                            b = copy.deepcopy(block_two)
                            while(a.below_of != None and b.below_of != None):
                                a = a.below_of
                                b = b.below_of
                                dic[(a.block_id, b.block_id)] = 1
    for x in goalState:
        count = 0
        ary = x.replace("(", "").replace(")", "").split()
        if(len(x.split()) == 4):
            if(ary[0] == "is"):
                if(ary[3] == "side-by-side"):
                    for y in goalState:
                        ary2 = y.replace("(", "").replace(")", "").split()
                        if(len(y.split()) == 4):
                            if(ary2[0] == "is"):
                                if(ary2[3] == "on-top-of"):
                                    if(ary[1] == ary2[1] and ary[2] != ary2[1]):
                                        count = count + 1
                                        block_one = next((g for g in state if g.block_id == ary[2]), None)
                                    if(ary[1] != ary2[1] and ary[2] == ary2[1]):
                                        count = count + 1
                                        block_one = next((g for g in state if g.block_id == ary[1]), None)
                                    
                    if(count == 1 and (block_one.below_of == None)):
                        cost = cost + 1
                        while(block_one.on_top_of != None and dic2.get((block_one.on_top_of.block_id, block_one.block_id), 0) == 0):
                            cost = cost + 1
                            dic2[(block_one.on_top_of.block_id, block_one.block_id)] = 1
                            block_one = block_one.on_top_of
                    
                            
    return cost

In [684]:
def h_7(state, goal):
    cost = 0
    dic = {} 
    dic2 = {}
    dic3 = {}
    for x in goalState:
        ary = x.replace("(", "").replace(")", "").split()
        if(len(x.split()) == 4):
            if(ary[0] == "is"):
                if(ary[3] == "on-top-of"):
                    block_one = next((g for g in state if g.block_id == ary[1]), None)
                    block_two = next((f for f in state if f.block_id == ary[2]), None)
                    if(dic2.get((ary[1], ary[2]), 0) == 0 and (block_one.below_of.block_id if block_one.below_of != None else None) != (ary[2] if ary[2] != 'table' else None)):
                        cost = cost + 1
                        dic2[(ary[1], block_one.below_of.block_id if block_one.below_of != None else None)] = 1
                        while(block_one.on_top_of != None and dic2.get((block_one.on_top_of.block_id, block_one.block_id), 0) == 0):
                            cost = cost + 1
                            dic2[(block_one.on_top_of.block_id, block_one.block_id)] = 1
                            block_one = block_one.on_top_of
                        while(block_two.on_top_of != None and dic2.get((block_two.on_top_of.block_id, block_two.block_id), 0) == 0):
                            cost = cost + 1
                            dic2[(block_two.on_top_of.block_id, block_two.block_id)] = 1
                            block_two = block_two.on_top_of
                        
    for x in goalState:
        ary = x.replace("(", "").replace(")", "").split()
        if(len(x.split()) == 4):
            if(ary[0] == "is"):
                if(ary[3] == "side-by-side"):
                    block_one = next((g for g in state if g.block_id == ary[1]), None)
                    block_two = next((f for f in state if f.block_id == ary[2]), None)
                    if(block_one.block_id not in [c.block_id for c in block_two.side_by_side]):
                        one = copy.deepcopy(block_one)
                        one_height = 0
                        two = copy.deepcopy(block_two)
                        two_height = 0
                        while(one.below_of != None):
                            one = one.below_of
                            one_height = one_height + 1
                        while(two.below_of != None):
                            two = two.below_of
                            two_height = two_height + 1
                        if(one_height != two_height):
                            cost = cost + 1
                            one = copy.deepcopy(block_one)
                            two = copy.deepcopy(block_two)
                            if(one_height == 0 and block_two.on_top_of == None and block_one.on_top_of != None):
                                cost = cost + 1
                            if(two_height == 0 and block_one.on_top_of == None and block_two.on_top_of != None):
                                cost = cost + 1
                            while(one.on_top_of != None and two.on_top_of != None and dic3.get((one.on_top_of.block_id, one.block_id), 0) == 0 and dic3.get((two.on_top_of.block_id, two.block_id), 0) == 0):
                                cost = cost + 1
                                dic3[(one.on_top_of.block_id, one.block_id)] = 1
                                dic3[(two.on_top_of.block_id, two.block_id)] = 1
                                one = one.on_top_of
                                two = two.on_top_of
                        elif(dic.get((ary[1], ary[2]), 0) == 0 and dic.get((ary[2], ary[1]), 0) == 0):
                            cost = cost + 1
                            dic[(ary[1], ary[2])] = 1
                            a = copy.deepcopy(block_one)
                            b = copy.deepcopy(block_two)
                            while(a.on_top_of != None and b.on_top_of != None):
                                a = a.on_top_of
                                b = b.on_top_of
                                
                                dic[(a.block_id, b.block_id)] = 1 
                            a = copy.deepcopy(block_one)
                            b = copy.deepcopy(block_two)
                            while(a.below_of != None and b.below_of != None):
                                a = a.below_of
                                b = b.below_of
                                dic[(a.block_id, b.block_id)] = 1
    for x in goalState:
        count = 0
        ary = x.replace("(", "").replace(")", "").split()
        if(len(x.split()) == 4):
            if(ary[0] == "is"):
                if(ary[3] == "side-by-side"):
                    for y in goalState:
                        ary2 = y.replace("(", "").replace(")", "").split()
                        if(len(y.split()) == 4):
                            if(ary2[0] == "is"):
                                if(ary2[3] == "on-top-of"):
                                    if(ary[1] == ary2[1] and ary[2] != ary2[1]):
                                        count = count + 1
                                        block_one = next((g for g in state if g.block_id == ary[2]), None)
                                    if(ary[1] != ary2[1] and ary[2] == ary2[1]):
                                        count = count + 1
                                        block_one = next((g for g in state if g.block_id == ary[1]), None)
                                    
                    if(count == 1 and (block_one.below_of == None)):
                        cost = cost + 1
                        while(block_one.on_top_of != None and dic2.get((block_one.on_top_of.block_id, block_one.block_id), 0) == 0):
                            cost = cost + 1
                            dic2[(block_one.on_top_of.block_id, block_one.block_id)] = 1
                            block_one = block_one.on_top_of


    return cost

In [794]:
def h_9(state, goal):
    cost = 0
    dic = {} 
    dic2 = {}
    for x in goalState:
        ary = x.replace("(", "").replace(")", "").split()
        if(len(x.split()) == 4):
            if(ary[0] == "is"):
                if(ary[3] == "on-top-of"):
                    block_one = next((g for g in state if g.block_id == ary[1]), None)
                    block_two = next((f for f in state if f.block_id == ary[2]), None)
                    if(dic2.get((ary[1], ary[2]), 0) == 0 and (block_one.below_of.block_id if block_one.below_of != None else None) != (ary[2] if ary[2] != 'table' else None)):
                        cost = cost + 1
                        dic2[(ary[1], block_one.below_of.block_id if block_one.below_of != None else None)] = 1
                        while(block_one.on_top_of != None and dic2.get((block_one.on_top_of.block_id, block_one.block_id), 0) == 0):
                            cost = cost + 1
                            dic2[(block_one.on_top_of.block_id, block_one.block_id)] = 1
                            block_one = block_one.on_top_of
                        while(block_two.on_top_of != None and dic2.get((block_two.on_top_of.block_id, block_two.block_id), 0) == 0):
                            cost = cost + 1
                            dic2[(block_two.on_top_of.block_id, block_two.block_id)] = 1
                            block_two = block_two.on_top_of
                        
    for x in goalState:
        ary = x.replace("(", "").replace(")", "").split()
        if(len(x.split()) == 4):
            if(ary[0] == "is"):
                if(ary[3] == "side-by-side"):
                    block_one = next((g for g in state if g.block_id == ary[1]), None)
                    block_two = next((f for f in state if f.block_id == ary[2]), None)
                    if(block_one.block_id not in [c.block_id for c in block_two.side_by_side]):
                        one = copy.deepcopy(block_one)
                        one_height = 0
                        two = copy.deepcopy(block_two)
                        two_height = 0
                        while(one.below_of != None):
                            one = one.below_of
                            one_height = one_height + 1
                        while(two.below_of != None):
                            two = two.below_of
                            two_height = two_height + 1
                        if(one_height != two_height):
                            cost = cost + 1
                            if(block_one.below_of != None and len(block_one.below_of.side_by_side) > 0):
                                cost = cost + 1
                            elif(block_two.below_of != None and len(block_two.below_of.side_by_side) > 0):
                                cost = cost + 1
                            one = copy.deepcopy(block_one)
                            two = copy.deepcopy(block_two)
                            while(one.on_top_of != None and two.on_top_of != None and dic2.get((one.on_top_of.block_id, one.block_id), 0) == 0 and dic2.get((two.on_top_of.block_id, two.block_id), 0) == 0):
                                cost = cost + 1
                                dic2[(one.on_top_of.block_id, one.block_id)] = 1
                                dic2[(two.on_top_of.block_id, two.block_id)] = 1
                                one = one.on_top_of
                                two = two.on_top_of
                        elif(dic.get((ary[1], ary[2]), 0) == 0 and dic.get((ary[2], ary[1]), 0) == 0):
                            cost = cost + 1
                            dic[(ary[1], ary[2])] = 1
                            a = copy.deepcopy(block_one)
                            b = copy.deepcopy(block_two)
                            while(a.on_top_of != None and b.on_top_of != None):
                                a = a.on_top_of
                                b = b.on_top_of
                                dic[(a.block_id, b.block_id)] = 1 
                            a = copy.deepcopy(block_one)
                            b = copy.deepcopy(block_two)
                            while(a.below_of != None and b.below_of != None):
                                a = a.below_of
                                b = b.below_of
                                dic[(a.block_id, b.block_id)] = 1
    for x in goalState:
        count = 0
        ary = x.replace("(", "").replace(")", "").split()
        if(len(x.split()) == 4):
            if(ary[0] == "is"):
                if(ary[3] == "side-by-side"):
                    for y in goalState:
                        ary2 = y.replace("(", "").replace(")", "").split()
                        if(len(y.split()) == 4):
                            if(ary2[0] == "is"):
                                if(ary2[3] == "on-top-of"):
                                    if(ary[1] == ary2[1] and ary[2] != ary2[1]):
                                        count = count + 1
                                        block_one = next((g for g in state if g.block_id == ary[2]), None)
                                    if(ary[1] != ary2[1] and ary[2] == ary2[1]):
                                        count = count + 1
                                        block_one = next((g for g in state if g.block_id == ary[1]), None)
                                    
                    if(count == 1 and (block_one.below_of == None)):
                        cost = cost + 1
                        while(block_one.on_top_of != None and dic2.get((block_one.on_top_of.block_id, block_one.block_id), 0) == 0):
                            cost = cost + 1
                            dic2[(block_one.on_top_of.block_id, block_one.block_id)] = 1
                            block_one = block_one.on_top_of
                    
                            
    return cost

In [795]:
startState = read_startState("input6.txt") #Input File
goalState = read_goalState("output6.txt") #Output File
print(startState)
print(h_9(startState, goalState))

[{block1, (color: red), (on-top-of: None), (side-by-side: ['block2', 'block3', 'block4', 'block5'])}
, {block10, (color: orange), (on-top-of: None), (side-by-side: [])}
, {block2, (color: green), (on-top-of: block6), (side-by-side: ['block1'])}
, {block3, (color: blue), (on-top-of: block7), (side-by-side: ['block1'])}
, {block4, (color: black), (on-top-of: block8), (side-by-side: ['block1'])}
, {block5, (color: orange), (on-top-of: block9), (side-by-side: ['block1'])}
, {block6, (color: pink), (on-top-of: None), (side-by-side: [])}
, {block7, (color: green), (on-top-of: None), (side-by-side: [])}
, {block8, (color: blue), (on-top-of: None), (side-by-side: [])}
, {block9, (color: black), (on-top-of: None), (side-by-side: [])}
]
3


In [588]:
def h_8(state, goal):
    cost = 0
    dic = {} 
    dic2 = {}
    dic3 = {}
    for x in goalState:
        ary = x.replace("(", "").replace(")", "").split()
        if(len(x.split()) == 4):
            if(ary[0] == "is"):
                if(ary[3] == "on-top-of"):
                    block_one = next((g for g in state if g.block_id == ary[1]), None)
                    block_two = next((f for f in state if f.block_id == ary[2]), None)
                    if(dic2.get((ary[1], ary[2]), 0) == 0 and (block_one.below_of.block_id if block_one.below_of != None else None) != (ary[2] if ary[2] != 'table' else None)):
                        cost = cost + 1
                        dic2[(ary[1], block_one.below_of.block_id if block_one.below_of != None else None)] = 1
                        while(block_one.on_top_of != None and dic2.get((block_one.on_top_of.block_id, block_one.block_id), 0) == 0):
                            cost = cost + 1
                            dic2[(block_one.on_top_of.block_id, block_one.block_id)] = 1
                            block_one = block_one.on_top_of
                        while(block_two.on_top_of != None and dic2.get((block_two.on_top_of.block_id, block_two.block_id), 0) == 0):
                            cost = cost + 1
                            dic2[(block_two.on_top_of.block_id, block_two.block_id)] = 1
                            block_two = block_two.on_top_of
                        
    for x in goalState:
        ary = x.replace("(", "").replace(")", "").split()
        if(len(x.split()) == 4):
            if(ary[0] == "is"):
                if(ary[3] == "side-by-side"):
                    block_one = next((g for g in state if g.block_id == ary[1]), None)
                    block_two = next((f for f in state if f.block_id == ary[2]), None)
                    if(block_one.block_id not in [c.block_id for c in block_two.side_by_side]):
                        cost = cost + 1
                        one = copy.deepcopy(block_one)
                        one_height = 0
                        two = copy.deepcopy(block_two)
                        two_height = 0
                        while(one.below_of != None):
                            one = one.below_of
                            one_height = one_height + 1
                        while(two.below_of != None):
                            two = two.below_of
                            two_height = two_height + 1
                        cost = cost + one_height
                        cost = cost + two_height
#                         if(one_height != two_height):
#                             cost = cost + 1
#                             cost = cost + one_height
#                             cost = cost + two_height
#                             one = copy.deepcopy(block_one)
#                             two = copy.deepcopy(block_two)
#                             if(one_height == 0 and block_two.on_top_of == None and block_one.on_top_of != None):
#                                 cost = cost + 1
#                             if(two_height == 0 and block_one.on_top_of == None and block_two.on_top_of != None):
#                                 cost = cost + 1
#                             while(one.on_top_of != None and two.on_top_of != None):
#                                 cost = cost + 1
#                                 dic2[(one.on_top_of.block_id, one.block_id)] = 1
#                                 dic2[(two.on_top_of.block_id, two.block_id)] = 1
#                                 one = one.on_top_of
#                                 two = two.on_top_of
#                         else:
#                             cost = cost + 1
#                             cost = cost + two_height
#                             dic[(ary[1], ary[2])] = 1
#                             a = copy.deepcopy(block_one)
#                             b = copy.deepcopy(block_two)
#                             while(a.on_top_of != None and b.on_top_of != None):
#                                 a = a.on_top_of
#                                 b = b.on_top_of
#                                 dic[(a.block_id, b.block_id)] = 1 
#                             a = copy.deepcopy(block_one)
#                             b = copy.deepcopy(block_two)
#                             while(a.below_of != None and b.below_of != None):
#                                 a = a.below_of
#                                 b = b.below_of
#                                 dic[(a.block_id, b.block_id)] = 1
#     for x in goalState:
#         count = 0
#         ary = x.replace("(", "").replace(")", "").split()
#         if(len(x.split()) == 4):
#             if(ary[0] == "is"):
#                 if(ary[3] == "side-by-side"):
#                     for y in goalState:
#                         ary2 = y.replace("(", "").replace(")", "").split()
#                         if(len(y.split()) == 4):
#                             if(ary2[0] == "is"):
#                                 if(ary2[3] == "on-top-of"):
#                                     if(ary[1] == ary2[1] and ary[2] != ary2[1]):
#                                         count = count + 1
#                                         block_one = next((g for g in state if g.block_id == ary[2]), None)
#                                     if(ary[1] != ary2[1] and ary[2] == ary2[1]):
#                                         count = count + 1
#                                         block_one = next((g for g in state if g.block_id == ary[1]), None)
                                    
#                     if(count == 1 and (block_one.below_of == None)):
#                         cost = cost + 1
#                         while(block_one.on_top_of != None and dic2.get((block_one.on_top_of.block_id, block_one.block_id), 0) == 0):
#                             cost = cost + 1
#                             dic2[(block_one.on_top_of.block_id, block_one.block_id)] = 1
#                             block_one = block_one.on_top_of
                    
                            
    return cost

In [608]:
startState = read_startState("case4start.txt") #Input File
goalState = read_goalState("case4end.txt") #Output File
print(startState)
print(h_7(startState, goalState))
stack(startState[12], None)
print("(stack block7 table)")
print(startState[12].block_id)

print(h_7(startState, goalState))
stack(startState[11], None)
print("(stack block6 table)")
print(startState[11].block_id)

print(h_7(startState, goalState))
stack(startState[10], None)
print("(stack block5 table)")
print(startState[10].block_id)

print(h_7(startState, goalState))
slide_to(startState[13], startState[10])
print("(slide-to block8 block5)")
print(startState[13].block_id)
print(startState[10].block_id)

print(h_7(startState, goalState))
stack(startState[9], None)
print("(stack block4 table)")
print(startState[9].block_id)

print(h_7(startState, goalState))
stack(startState[8], None)
print("stack block3 table)")
print(startState[8].block_id)

print(h_7(startState, goalState))
stack(startState[7], None)
print("stack block2 table)")
print(startState[7].block_id)

print(h_7(startState, goalState))
slide_to(startState[7], startState[10])
print("(slide-to block2 block5)")
print(startState[7].block_id)
print(startState[10].block_id)

print(h_7(startState, goalState))
slide_to(startState[0], startState[7])
print("(slide-to block1 block2)")
print(startState[0].block_id)
print(startState[7].block_id)

print(h_7(startState, goalState))
slide_to(startState[9], startState[0])
print("(slide-to block4 block1)")
print(startState[9].block_id)
print(startState[0].block_id)

print(h_7(startState, goalState))
slide_to(startState[12], startState[7])
print("(slide-to block7 block2)")
print(startState[12].block_id)
print(startState[7].block_id)

print(h_7(startState, goalState))
slide_to(startState[8], startState[0])
print("((slide-to block3 block1)")
print(startState[8].block_id)
print(startState[0].block_id)

print(h_7(startState, goalState))
slide_to(startState[11], startState[7])
print("(slide-to block6 block2)")
print(startState[11].block_id)
print(startState[7].block_id)

print(h_7(startState, goalState))
stack(startState[5], startState[0])
print("(stack block14 block1)")
print(startState[5].block_id)
print(startState[0].block_id)

print(h_7(startState, goalState))
stack(startState[6], startState[8])
print("(stack block15 block3)")
print(startState[6].block_id)
print(startState[8].block_id)

print(h_7(startState, goalState))
stack(startState[4], None)
print("(stack block13 table)")
print(startState[4].block_id)

print(h_7(startState, goalState))
stack(startState[3], None)
print("(stack block12 table)")
print(startState[3].block_id)

print(h_7(startState, goalState))
stack(startState[2], None)
print("(stack block11 table)")
print(startState[2].block_id)

print(h_7(startState, goalState))
slide_to(startState[2], startState[8])
print("(slide-to block11 block3)")
print(startState[2].block_id)
print(startState[8].block_id)

print(h_7(startState, goalState))
stack(startState[4], startState[2])
print("(stack block13 block11)")
print(startState[4].block_id)
print(startState[2].block_id)

print(h_7(startState, goalState))
stack(startState[1], None)
print("(stack block10 table)")
print(startState[1].block_id)

print(h_7(startState, goalState))
slide_to(startState[1], startState[0])
print("(slide-to block10 block1)")
print(startState[1].block_id)
print(startState[0].block_id)

print(h_7(startState, goalState))
stack(startState[3], startState[1])
print("(stack block12 block10)")
print(startState[3].block_id)
print(startState[1].block_id)

print(h_7(startState, goalState))
stack(startState[14], None)
print("(stack block9 table)")
print(startState[14].block_id)

print(h_7(startState, goalState))
slide_to(startState[14], startState[10])
print("(slide-to block9 block5)")
print(startState[14].block_id)
print(startState[10].block_id)

print(h_7(startState, goalState))

[{block1, (color: red), (on-top-of: block2), (side-by-side: [])}
, {block10, (color: red), (on-top-of: block11), (side-by-side: [])}
, {block11, (color: blue), (on-top-of: block12), (side-by-side: [])}
, {block12, (color: blue), (on-top-of: block13), (side-by-side: [])}
, {block13, (color: green), (on-top-of: block14), (side-by-side: [])}
, {block14, (color: green), (on-top-of: None), (side-by-side: [])}
, {block15, (color: green), (on-top-of: None), (side-by-side: [])}
, {block2, (color: blue), (on-top-of: block3), (side-by-side: [])}
, {block3, (color: green), (on-top-of: block4), (side-by-side: [])}
, {block4, (color: black), (on-top-of: block5), (side-by-side: [])}
, {block5, (color: white), (on-top-of: block6), (side-by-side: [])}
, {block6, (color: yellow), (on-top-of: block7), (side-by-side: [])}
, {block7, (color: yellow), (on-top-of: None), (side-by-side: [])}
, {block8, (color: yellow), (on-top-of: block9), (side-by-side: [])}
, {block9, (color: red), (on-top-of: block10), (s

In [550]:
startState = read_startState("case3bstart.txt") #Input File
print(startState)
print(h_6(startState, goalState))
goalState = read_goalState("case3bend.txt") #Output File
stack(startState[0], None)
print(h_6(startState, goalState))
stack(startState[1], startState[0])
print(h_6(startState, goalState))
stack(startState[2], None)
print(h_6(startState, goalState))
stack(startState[3], startState[2])
print(h_6(startState, goalState))
slide_to(startState[0], startState[2])
print(h_6(startState, goalState))

[{block1, (color: red), (on-top-of: None), (side-by-side: [])}
, {block2, (color: blue), (on-top-of: block1), (side-by-side: [])}
, {block3, (color: green), (on-top-of: None), (side-by-side: [])}
, {block4, (None: None), (on-top-of: block3), (side-by-side: [])}
]


AttributeError: 'NoneType' object has no attribute 'below_of'

In [61]:
startState = read_startState("input2.txt") #Input File
print(startState)
goalState = read_goalState("output2.txt") #Output File
print(goalState)
slide_to(startState[5], startState[0])
print(h_6(startState, goalState))
stack(startState[4], 'table')
print(h_6(startState, goalState))
slide_to(startState[4], startState[0])
print(h_6(startState, goalState))
stack(startState[2], 'table')
print(h_6(startState, goalState))
slide_to(startState[2], startState[0])
print(h_6(startState, goalState))
stack(startState[6], startState[0])
print(h_6(startState, goalState))
stack(startState[7], startState[6])
print(h_6(startState, goalState))
stack(startState[8], startState[7])
print(h_6(startState, goalState))
stack(startState[9], startState[8])
print(h_6(startState, goalState))
stack(startState[1], startState[9])
print(h_6(startState, goalState))

[{block1, (color: red), (on-top-of: block2), (side-by-side: ['block3'])}
, {block10, (None: None), (on-top-of: None), (side-by-side: [])}
, {block2, (None: None), (on-top-of: None), (side-by-side: ['block4'])}
, {block3, (color: blue), (on-top-of: block4), (side-by-side: ['block1'])}
, {block4, (None: None), (on-top-of: None), (side-by-side: ['block2'])}
, {block5, (color: green), (on-top-of: block10), (side-by-side: ['block6', 'block7', 'block8', 'block9'])}
, {block6, (None: None), (on-top-of: None), (side-by-side: ['block5'])}
, {block7, (color: yellow), (on-top-of: None), (side-by-side: ['block5'])}
, {block8, (None: None), (on-top-of: None), (side-by-side: ['block5'])}
, {block9, (color: purple), (on-top-of: None), (side-by-side: ['block5'])}
]
['(is block1 block2 side-by-side)\n', '(is block1 block3 side-by-side)\n', '(is block1 block4 side-by-side)\n', '(is block1 block5 side-by-side)\n', '(is block6 block1 on-top-of)\n', '(is block7 block6 on-top-of)\n', '(is block8 block7 on-t

Node Class: h is heuristic cost of state, g is distance from startState, f is cost from startState

In [62]:
class Node:
    def __init__(self, state, action, h=0, g=0 ,f=0):
        self.state = state
        self.action = action
        self.h = h
        self.g = g
        self.f = f
    def __repr__(self):
        return "Node(" + repr(self.state) + ", action=" + repr(self.action) + ", f=" + repr(self.f) + \
               ", g=" + repr(self.g) + ", h=" + repr(self.h) + ")"
    def __str__(self):
        return "Node(" + repr(self.state) + ", action=" + repr(self.action) + ", f=" + repr(self.f) + \
               ", g=" + repr(self.g) + ", h=" + repr(self.h) + ")"

A\* Search

In [679]:
def aStar(parentNode, goalState, actionsF, take_actionF, hF, fmax):
    if(check(parentNode.state, goalState)):
        return([parentNode.action], parentNode.g)
    actions = actionsF(parentNode.state)
    if not actions:
        return("no more moves", float("inf"))
    children = []
    for action in actions:
        childState = take_actionF(parentNode.state, action)
        h = hF(childState, goalState)
        g = parentNode.g + 1
        f = max(h+g, parentNode.f)
        childNode = Node(state=childState, action = action, h=h, g=g, f=f)
        children.append(childNode)
    while True:
        minChild = min(children, key = lambda x: x.f)
        if minChild.f > fmax:
            return ("max steps exceded", minChild.f)
        alternativef = minChild.f if len(children) > 1 else float('inf')
        print(minChild.g)
        print("heuristic " + str(h_7(minChild.state, goalState)))
        result, minChild.f = aStar(minChild, goalState, actionsF, take_actionF, hF, min(fmax,alternativef))
        if result is not "no more moves" and result is not "max steps exceded":         
            result.insert(0, parentNode.action) 
            return (result, minChild.f) 
        
def aStarSearch(startState, goalState, actionsF, take_actionF, hF, fmax):
    return aStar(Node(state=startState, action=None, f=0, g=0, h=0), goalState, actionsF, take_actionF, hF, fmax)

In [793]:
startState = read_startState("case4start.txt") #Input File
goalState = read_goalState("case4end.txt") #Output File
print(startState)
print(goalState)
print()
answer = aStarSearch(startState, goalState, actions, take_action, h_9, 1000)
if((answer[0] ==  "max steps exceded") or (answer[0] == "no more moves")):
    print(answer)
else:
    for a in answer[0][1::]:
        print("(" + str(a[0]) + ", " + str(a[1][1]) + ", " + (str(a[2][1]) if a[2] != None else "table") + ")")
    print(answer[1])

[{block1, (color: red), (on-top-of: block2), (side-by-side: [])}
, {block10, (color: red), (on-top-of: block11), (side-by-side: [])}
, {block11, (color: blue), (on-top-of: block12), (side-by-side: [])}
, {block12, (color: blue), (on-top-of: block13), (side-by-side: [])}
, {block13, (color: green), (on-top-of: block14), (side-by-side: [])}
, {block14, (color: green), (on-top-of: None), (side-by-side: [])}
, {block15, (color: green), (on-top-of: None), (side-by-side: [])}
, {block2, (color: blue), (on-top-of: block3), (side-by-side: [])}
, {block3, (color: green), (on-top-of: block4), (side-by-side: [])}
, {block4, (color: black), (on-top-of: block5), (side-by-side: [])}
, {block5, (color: white), (on-top-of: block6), (side-by-side: [])}
, {block6, (color: yellow), (on-top-of: block7), (side-by-side: [])}
, {block7, (color: yellow), (on-top-of: None), (side-by-side: [])}
, {block8, (color: yellow), (on-top-of: block9), (side-by-side: [])}
, {block9, (color: red), (on-top-of: block10), (s

KeyboardInterrupt: 